In [9]:
from datetime import datetime
from sagemaker.pytorch.estimator import PyTorch
import os
import boto3
import pandas as pd
import sagemaker

In [ ]:
def main_pipeline(dataset, n_folds,dict1, eval_test, label, 
                  model_name, model_alias, epochs,
                  batch_size, seed_val,  verbose):
    kit_label_encoding_cresemba = open_util('sbx-test-12343977','utility_files/cresemba_KITS.txt','txt')
    kit_label_decoding_cresemba = dict([(value, key) for key, value in kit_label_encoding_cresemba.items()])

    if label == 'KIT description':
        if dataset == 'cresemba':
            label_encoding = kit_label_encoding_cresemba
            
            dataset_df = preprocess_cresemba_raw_for_kit(dict1)
        elif dataset == 'zavicefta':
            label_encoding = kit_label_encoding_zavicefta
            
            dataset_df = preprocess_zavicefta_raw_for_kit()
    elif label == 'Insights Category':
        if dataset == 'cresemba':
            label_encoding = insights_label_encoding_cresemba
            
            dataset_df = preprocess_cresemba_raw_for_insights(label_encoding)
        elif dataset == 'zavicefta':
            label_encoding = insights_label_encoding_zavicefta
            
            dataset_df = preprocess_zavicefta_raw_for_insights(label_encoding)

    labels_set = list(label_encoding.keys())
    
    
    biobert_models_path = os.path.join(*[project_path, "data/biobert_model_files"])

    cv_data, _ = cross_val(df=dataset_df, n_folds=int(n_folds), label=label)

    print('Number of data points in each class:')
    labels = cv_data[0][0][label]
    print(Counter(labels))

    cv_data, label_set = pick_data_for_exp_scenario(folds=cv_data, eval_test=eval_test, dataset=dataset, label=label,
                                                    label_mapping=label_encoding, labels_set=labels_set)

    

    best_predictions_list = cross_val_train(cv_data, model_name,
                                            epochs, batch_size, seed_val, 
                                            verbose, biobert_models_path, model_alias)

    metrics = eval_avg_score(best_predictions_list, n_folds, project_path, label, dataset)

In [ ]:
if __name__ == "__main__":

    main_pipeline(dataset = "cresemba",
                    n_folds = 2,
                    eval_test = True,
                    label = "KIT description",
                    
                    # DEFINE PARAMETERS FOR FINETUNING:
                    model_name = 'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract',
                    model_alias = 'ms_pubmed_bert', #for model saving file name
                    epochs = 1,
                    batch_size = 3,
                    seed_val = 17,
                    dict1=open_util('sbx-test-12343977','Medical Insights_May2021_DFOs_SV_Cresemba_(Isavuconazole).xlsx',file_type='xlsx'),
                    verbose = False,
                    cresembafile=
                    
                    
                    )
    torch.cuda.empty_cache()

In [7]:
def read_object(s3_path, file_type, **kwargs):
    """Read a file from S3 into a pandas.DataFrame.
    Arguments:
        s3_path (str)
    Returns:
        df (pd.DataFrame)
    """
    # Get S3 client.
    s3 = boto3.client('s3')
    # Get object.
    bucket, key = s3_path.split('/')[2], '/'.join(s3_path.split('/')[3:])
    object = s3.get_object(Bucket=bucket, Key=key)
    # Read object into pandas.DataFrame.
    if file_type == "csv":
        df = pd.read_csv(object['Body'], **kwargs)
    elif file_type == "json":
        df = pd.read_json(object['Body'], lines=True, **kwargs)
    elif file_type == "xlsx":
        file_content = object["Body"].read()
        read_excel_data = io.BytesIO(file_content)
        df = pd.read_excel(read_excel_data, engine="openpyxl", sheet_name="MEDIC_OneMed_DFO_Isavuconazole", skiprows=1,
                         keep_default_na=False)
        
    else:
        raise Exception("Only file_type csv and json currently supported.")
    return df

In [3]:
import pandas as pd
import io

In [5]:
file=f"s3://sbx-test-12343977/Medical Insights_May2021_DFOs_SV_Cresemba_(Isavuconazole).xlsx"

In [8]:
read_object(file, 'xlsx')

,CFM Form ID,DFO Description,KIT description,Insights Category,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,kit,insights
0,178762,there is an unmet need in fungi diagnostics an...,Awareness & diagnosis of invasive fungal infec...,Awareness,,,,,,,Cresemba,1,0
1,184389,when presenting isavuconazole data in critical...,Awareness & diagnosis of invasive fungal infec...,Awareness,,,,,,,zavicefta,1,0
2,198269,importance of following patients with severe c...,Awareness & diagnosis of invasive fungal infec...,Awareness,,,,,,,,,
3,226329,"in paediatric patients, there are difficulties...",Awareness & diagnosis of invasive fungal infec...,Awareness,,,,,,,,,
4,242516,3 haematooncology centers in poland have exper...,Awareness & diagnosis of invasive fungal infec...,Awareness,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1731,310451,"""the doctor sees 5-10 cases of mm per year and...",Not Applicable,,,,,,,,,,
1732,303623,is there are correlation between construction ...,Resistance & epidemiology of IFI,,,,,,,,,,
1733,310431,"""the doctor sees 6-10 cases of mm per year and...",Not Applicable,,,,,,,,,,
1734,310440,"""the doctor sees 5-10 cases of mm per year and...",Not Applicable,,,,,,,,,,
